🎌 WhisperJAV - Google Colab Edition
Generate Accurate Subtitles for Japanese Adult Videos using Free GPU

📋 **Quick Start Guide**
1.  Run **Cell 1** → Connect Google Drive
2.  Run **Cell 2** → Install WhisperJAV
3.  Run **Cell 3** → Launch Control Panel
4.  Select Options → Click "Start Processing"
5.  Check Results → In your Drive's `WhisperJAV` folder

⚡ **Pro Tip:** Use `Runtime` → `Run all` to setup everything at once!

In [1]:
#@title 3️⃣ Configure WhisperJAV Options
#@markdown Select your desired settings for processing. These will be used when you run the next cell.

!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, HTML

# --- Custom CSS for better styling ---
display(HTML("""
<style>
.whisperjav-container {
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
}
.control-section {
    background: white;
    border: 1px solid #e1e4e8;
    border-radius: 8px;
    padding: 20px;
    margin: 10px 0;
    box-shadow: 0 1px 3px rgba(0,0,0,0.1);
}
.section-title {
    font-size: 18px;
    font-weight: 600;
    color: #24292e;
    margin-bottom: 15px;
    display: flex;
    align-items: center;
}
.section-title .emoji {
    margin-right: 8px;
    font-size: 24px;
}
</style>
"""))

# --- Main UI Container ---
main_container = widgets.VBox()
display(HTML('<div class="whisperjav-container">'))

# Title
title_html = HTML("""
<h1 style="text-align: center; color: #24292e; margin: 20px 0;">
    🎌 WhisperJAV Control Panel
</h1>
""")

# --- Speed Control Section ---
speed_title = HTML("""
<div class="control-section">
    <div class="section-title">
        <span class="emoji">⚡</span>
        Speed Control
        <span style="font-size: 14px; color: #586069; margin-left: 10px;">
            (Quickie vs Less Mistakes)
        </span>
    </div>
""")
mode_widget = widgets.RadioButtons(
    options=[
        ('🚀 Faster - Quick processing, may miss some details', 'faster'),
        ('⚡ Fast - Good balance of speed and accuracy', 'fast'),
        ('🎯 Balanced - Best accuracy, takes more time', 'balanced')
    ],
    value='balanced',
    layout=widgets.Layout(width='100%')
)
speed_box = widgets.VBox([speed_title, mode_widget])


# --- Granularity Control Section ---
granularity_title = HTML("""
<div class="control-section">
    <div class="section-title">
        <span class="emoji">🔍</span>
        Granularity Control
        <span style="font-size: 14px; color: #586069; margin-left: 10px;">
            (Details vs Fewer Guesses)
        </span>
    </div>
""")
sensitivity_widget = widgets.RadioButtons(
    options=[
        ('🔥 Aggressive - Capture everything, more false positives', 'aggressive'),
        ('⚖️ Balanced - Optimal for most content', 'balanced'),
        ('🎯 Conservative - Only high confidence, may miss some', 'conservative')
    ],
    value='balanced',
    layout=widgets.Layout(width='100%')
)
granularity_box = widgets.VBox([granularity_title, sensitivity_widget])

# --- Language Section ---
language_title = HTML("""
<div class="control-section">
    <div class="section-title">
        <span class="emoji">🌐</span>
        Output Language
    </div>
""")
language_widget = widgets.RadioButtons(
    options=[
        ('🇯🇵 Japanese - Original transcription', 'japanese'),
        ('🇬🇧 English - Direct translation', 'english-direct')
    ],
    value='japanese',
    layout=widgets.Layout(width='100%')
)
language_box = widgets.VBox([language_title, language_widget])

# --- Enhancement Options Section ---
enhancements_title = HTML("""
<div class="control-section">
    <div class="section-title">
        <span class="emoji">✨</span>
        Enhancement Options
    </div>
""")
adaptive_classification_widget = widgets.Checkbox(
    value=False, description='Adaptive Classification', style={'description_width': 'initial'}
)
adaptive_audio_widget = widgets.Checkbox(
    value=False, description='Adaptive Audio Enhancement', style={'description_width': 'initial'}
)
smart_post_widget = widgets.Checkbox(
    value=False, description='Smart Postprocessing', style={'description_width': 'initial'}
)
enhancements_box = widgets.VBox(
    [enhancements_title, adaptive_classification_widget, adaptive_audio_widget, smart_post_widget]
)


# --- Display all UI sections ---
main_container.children = [
    title_html,
    speed_box,
    granularity_box,
    language_box,
    enhancements_box
]

display(main_container)
display(HTML('</div>'))

print("✅ Configuration panel is ready. Make your selections and then run the next cell to start processing.")

In [ ]:
#@title 4️⃣ Start Processing
#@markdown This will run WhisperJAV using the options you selected in the panel above.

import shlex
from pathlib import Path

# --- Define the target folder in Google Drive ---
# This path should be set in a previous cell where the user connects their drive.
drive_folder = Path('/content/drive/MyDrive/WhisperJAV')
output_folder = drive_folder

# --- Build the command from the widget selections of the previous cell ---
base_cmd = f"python3 -m whisperjav '{drive_folder}'"
options = [
    f"--mode {shlex.quote(mode_widget.value)}",
    f"--sensitivity {shlex.quote(sensitivity_widget.value)}",
    f"--subs-language {shlex.quote(language_widget.value)}",
    f"--output-dir '{output_folder}'"
]

# Add boolean flags only if they are checked
if adaptive_classification_widget.value:
    options.append("--adaptive-classification")
if adaptive_audio_widget.value:
    options.append("--adaptive-audio-enhancement")
if smart_post_widget.value:
    options.append("--smart-postprocessing")

# Join all parts of the command into a single string
full_command = f"{base_cmd} {' '.join(options)}"

print("▶️ Starting WhisperJAV with the following command:")
print(full_command)
print("-" * 50)

# --- Execute the command ---
!{full_command}

print("-" * 50)
print("✅ Processing complete!")

In [ ]:
#@markdown Mount your Google Drive to access media files

from google.colab import drive
import os
from pathlib import Path
import IPython.display as display

print("🔄 Connecting to Google Drive...")
drive.mount('/content/drive')

# Create WhisperJAV folder if it doesn't exist
whisperjav_folder = Path('/content/drive/MyDrive/WhisperJAV')
whisperjav_folder.mkdir(exist_ok=True)

# Check for media files
media_extensions = ['.mp3',
                    '.wav',
                    '.opus',
                    '.m4a',
                    '.flac',
                    '.wmv',
                    '.mp4',
                    '.mkv',
                    '.webm']
media_files = []
for ext in media_extensions:
    media_files.extend(whisperjav_folder.glob(f'*{ext}'))

print("✅ Google Drive connected!")
print(f"📁 WhisperJAV folder: {whisperjav_folder}")
print(f"🎬 Found {len(media_files)} media file(s)")

if not media_files:
    display.display(display.HTML("""
    <div style=\"background-color: #fff3cd; border: 1px solid #ffeaa7; border-radius: 5px; padding: 15px; margin: 10px 0;\">
        <h4 style=\"color: #856404; margin: 0;\">⚠️ No media files found!</h4>
        <p style=\"margin: 5px 0;\">Please upload your media files to:<br>
        <code style=\"background: #f8f9fa; padding: 2px 5px; border-radius: 3px;\">My Drive/WhisperJAV/</code></p>
        <p style=\"margin: 5px 0; font-size: 0.9em;\">Supported formats: MP4, AVI, MKV, MOV, WMV, FLV, WEBM</p>
    </div>
    """))
else:
    display.display(display.HTML("""
    <div style=\"background-color: #d4edda; border: 1px solid #c3e6cb; border-radius: 5px; padding: 15px; margin: 10px 0;\">
        <h4 style=\"color: #155724; margin: 0;\">✅ Ready to process!</h4>
        <p style=\"margin: 5px 0;\">Found medias in your WhisperJAV folder</p>
    </div>
    """))

In [ ]:
#@markdown This will install all required packages (takes ~2-3 minutes)

print("📦 Installing WhisperJAV and dependencies...")
print("This may take a few minutes on first run...\n")


# --- Step 1: Install system dependencies ---
print("1/4 Installing FFmpeg...")
!apt-get update -qq && apt-get install -y -qq ffmpeg

# --- Step 2: Install Python packages ---
print("\n2/4 Installing PyTorch with GPU support...")
# pip installs
!pip install -q \
  torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124


# --- Step 3 & 4 are combined above ---
print("\n3/4 Installing WhisperJAV...")
print("4/4 Installing UI components...")

!pip install    git+https://github.com/openai/whisper.git
!pip install    "faster-whisper @ https://github.com/SYSTRAN/faster-whisper/archive/refs/heads/master.tar.gz"
!pip install -U git+https://github.com/jianfch/stable-ts.git
!pip install -U git+https://github.com/meizhong986/WhisperJAV.git
!pip install    ipywidgets


print("\n✅ Installation complete!")

# --- Final Check: Verify GPU ---
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🎮 GPU detected: {gpu_name}")
else:
    print("⚠️ No GPU detected - processing will be slower")

In [ ]:
#@markdown Check your generated subtitles and download if needed

import os
import zipfile
from datetime import datetime
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path

whisperjav_folder = Path('/content/drive/MyDrive/WhisperJAV')
# List all SRT files
srt_files = list(whisperjav_folder.glob('*.srt'))
raw_subs_folder = whisperjav_folder / 'raw_subs'
raw_files = list(raw_subs_folder.glob('*')) if raw_subs_folder.exists() else []

print("📄 Subtitle Files Generated:")
print("-" * 50)

if srt_files:
    for srt in sorted(srt_files):
        size_kb = srt.stat().st_size / 1024
        print(f"✅ {srt.name} ({size_kb:.1f} KB)")
else:
    print("❌ No subtitle files found yet. Please run processing first.")

if raw_files:
    print(f"\n📁 Raw outputs: {len(raw_files)} file(s) in raw_subs/")

# Create download button
if srt_files or raw_files:
    print("\n" + "="*50)

    # Create a download widget
    download_button = widgets.Button(
        description='Download All Subtitles as ZIP',
        icon='download',
        button_style='primary',
        layout=widgets.Layout(width='300px')
    )
    download_output = widgets.Output()

    def download_all(b):
        with download_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            zip_name = f'WhisperJAV_subtitles_{timestamp}.zip'
            zip_path = Path(f'/content/{zip_name}')

            print(f"Creating {zip_name}...")
            with zipfile.ZipFile(zip_path, 'w') as zipf:
                # Add main SRT files
                for srt in srt_files:
                    zipf.write(srt, srt.name)

                # Add raw files
                for raw in raw_files:
                    zipf.write(raw, f'raw_subs/{raw.name}')

            files.download(str(zip_path))
            print(f"✅ Download started for {zip_name}")
            # os.remove(zip_path)

    download_button.on_click(download_all)
    display(download_button, download_output)

    # Show file preview option
    if srt_files:
        print("\n📖 Preview first few lines of latest subtitle:")
        latest_srt = max(srt_files, key=lambda x: x.stat().st_mtime)
        try:
            with open(latest_srt, 'r', encoding='utf-8') as f:
                preview = f.read(500)
                print(f"\n--- {latest_srt.name} ---")
                print(preview)
                if len(f.read()) > 0:
                    print("... (truncated)")
        except Exception as e:
            print(f"Could not read file: {e}")

In [ ]:
#@markdown View GPU and system status

import subprocess
print("🎮 GPU Info:")
!nvidia-smi --query-gpu=name,memory.total,memory.used,temperature.gpu --format=csv

print("\n" + "="*50)

# Disk usage
print("💾 Disk Usage:")
!df -h | grep -E "Filesystem|/content"

print("\n" + "="*50)

# WhisperJAV folder size
whisperjav_folder_path = '/content/drive/MyDrive/WhisperJAV'

result = subprocess.run(['du', '-sh', whisperjav_folder_path], capture_output=True, text=True)
print(f"📁 WhisperJAV folder size: {result.stdout.strip()}")

print("\n" + "="*50)

# Python packages
print("📦 Installed Packages:")
!pip list | grep -E "whisperjav|whisper|torch"

💡 **Tips & Troubleshooting**
---
**🚀 Performance Tips:**
- `Faster` mode: Best for quick previews (5-10x faster)
- `Balanced` mode: Recommended for final subtitles
- **GPU**: A T4 GPU (usually available in the free tier) provides a ~10x speedup over CPU.

**❓ Common Issues:**
- **No media files found?**
  - Ensure you've uploaded medias to: `My Drive/WhisperJAV/`
  - Re-run Cell 1 and Cell 3 to refresh the file list.
- **Out of memory?**
  - Try using the `faster` mode, which uses a smaller model.
  - Process shorter medias individually.
  - Restart the runtime: `Runtime` → `Restart runtime`
- **Session disconnected?**
  - Colab free tier has time limits and will disconnect due to inactivity.
  - Keep the browser tab active to prevent automatic disconnection.

**📞 Support:**
- Report issues: [GitHub Issues](https://github.com/meizhong986/whisperjav/issues)
- Documentation: [WhisperJAV Docs](https://github.com/meizhong986/whisperjav)

*Made with ❤️ for the JAV subtitle community*